#Preprocessing 1
In the original dataset, there are some features have little impacts on whether the traffic is abnormal or not, such as timestamps and IP addresses. The timestamp records the time when the anomalous network traffic occurred, which are of little help in training our neural network, so we removed this feature. In addition, as an anomaly detection system, we hope it can classify the network traffics according to their behavioral characteristics, and should not be biased against the IP address, so we also deleted the column of feature.

**So we under-sampled the normal traffics and only took 2 million records randomly**


---

In [1]:
import pandas as pd
from csv import reader
pd.set_option('display.max_columns', 50)
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import random

In [2]:
#muestraDF.to_csv('DF_undersample.csv', index = False)


muestraDF=pd.read_csv("DF_undersample.csv",sep=',')
print (pd.unique(muestraDF.iloc[:,-1].squeeze()))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['Benign' 'Bot' 'DoS attacks-SlowHTTPTest' 'DoS attacks-Hulk' 'Label'
 'Brute Force -Web' 'Brute Force -XSS' 'SQL Injection'
 'DoS attacks-GoldenEye' 'DoS attacks-Slowloris' 'FTP-BruteForce'
 'SSH-Bruteforce' 'DDOS attack-LOIC-UDP' 'DDOS attack-HOIC'
 'Infilteration' 'DDoS attacks-LOIC-HTTP']


In [3]:
print(muestraDF.dtypes)

Dst Port          object
 Protocol         object
 Timestamp        object
 Flow Duration    object
 Tot Fwd Pkts     object
                   ...  
Idle Mean         object
Idle Std          object
Idle Max          object
Idle Min          object
Label             object
Length: 80, dtype: object


#Preprocessing 2

After completing the above works, we divide the dataset into training set, test set and validation set, which are 90%, 9% and 1% of the original data respectively. The training set is used for training, the validation set is used for rapid evaluation of the model during training, and the test set is used for final evaluation of the model. In addition, we noticed that there are too many normal network traffic samples in the dataset, which can easily affect the classification preference of the model. **So we under-sampled the normal traffics and only took 2 million records randomly**. 

In [4]:
print(muestraDF.shape)
print(muestraDF.isna().values.any())
muestraDF=muestraDF.drop(columns=[' Timestamp'],axis=1)
print(muestraDF.shape)

(4096753, 80)
True
(4096753, 79)


In [5]:
y=muestraDF.iloc[:,-1]
print (pd.unique(y.squeeze()))

['Benign' 'Bot' 'DoS attacks-SlowHTTPTest' 'DoS attacks-Hulk' 'Label'
 'Brute Force -Web' 'Brute Force -XSS' 'SQL Injection'
 'DoS attacks-GoldenEye' 'DoS attacks-Slowloris' 'FTP-BruteForce'
 'SSH-Bruteforce' 'DDOS attack-LOIC-UDP' 'DDOS attack-HOIC'
 'Infilteration' 'DDoS attacks-LOIC-HTTP']


In [6]:
X=muestraDF.iloc[:,0:78]
X = X.replace([np.inf, -np.inf], np.nan)
nom_cols=['Dst Port',' Protocol',' Flow Duration',' Tot Fwd Pkts',' Tot Bwd Pkts',' TotLen Fwd Pkts',' TotLen Bwd Pkts',' Fwd Pkt Len Max',' Fwd Pkt Len Min',' Fwd Pkt Len Mean',' Fwd Pkt Len Std',' Bwd Pkt Len Max',' Bwd Pkt Len Min',' Bwd Pkt Len Mean',' Bwd Pkt Len Std',' Flow Byts/s',' Flow Pkts/s',' Flow IAT Mean',' Flow IAT Std',' Flow IAT Max',' Flow IAT Min',' Fwd IAT Tot',' Fwd IAT Mean',' Fwd IAT Std',' Fwd IAT Max',' Fwd IAT Min',' Bwd IAT Tot',' Bwd IAT Mean',' Bwd IAT Std',' Bwd IAT Max',' Bwd IAT Min',' Fwd PSH Flags',' Bwd PSH Flags',' Fwd URG Flags',' Bwd URG Flags',' Fwd Header Len',' Bwd Header Len',' Fwd Pkts/s',' Bwd Pkts/s',' Pkt Len Min',' Pkt Len Max',' Pkt Len Mean',' Pkt Len Std',' Pkt Len Var',' FIN Flag Cnt',' SYN Flag Cnt',' RST Flag Cnt',' PSH Flag Cnt',' ACK Flag Cnt',' URG Flag Cnt',' CWE Flag Count',' ECE Flag Cnt',' Down/Up Ratio',' Pkt Size Avg',' Fwd Seg Size Avg',' Bwd Seg Size Avg',' Fwd Byts/b Avg',' Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg',' Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts',' Subflow Bwd Pkts', 'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std', 'Active Max',' Active Min', 'Idle Mean', 'Idle Std', 'Idle Max','Idle Min']
for i in nom_cols:
    X[i]=pd.to_numeric(X[i], errors='coerce')
    X[i] = X[i].fillna(X[i].mean())
print(X.isna().values.any())
print(X.dtypes)
X.index[np.isinf(X).any(1)]

False
Dst Port          float64
 Protocol         float64
 Flow Duration    float64
 Tot Fwd Pkts     float64
 Tot Bwd Pkts     float64
                   ...   
 Active Min       float64
Idle Mean         float64
Idle Std          float64
Idle Max          float64
Idle Min          float64
Length: 78, dtype: object


Int64Index([     66,     105,     110,     116,   32494,   32513,   43093,
              43128,   43137,   43138,
            ...
            4085176, 4085198, 4085904, 4087502, 4087517, 4087536, 4087881,
            4089289, 4089296, 4089457],
           dtype='int64', length=10620)

In [7]:
X = X.replace([np.inf, -np.inf], np.nan)
for i in nom_cols:
    X[i]=pd.to_numeric(X[i], errors='coerce')
    X[i] = X[i].fillna(X[i].mean())

X.index[np.isinf(X).any(1)]
print(X.isna().values.any())

False


In [8]:
X.index[np.isinf(X).any(1)]



Int64Index([], dtype='int64')

In [9]:
y=y.replace (['DDOS attack-LOIC-UDP', 'DDOS attack-HOIC','DDoS attacks-LOIC-HTTP'],'DDoS')
y=y.replace (['Brute Force -Web', 'Brute Force -XSS', 'SQL Injection'],'Web_attack')
y=y.replace (['FTP-BruteForce', 'SSH-Bruteforce'],'Bruteforce')
y=y.replace (['DoS attacks-GoldenEye', 'DoS attacks-Slowloris', 'DoS attacks-SlowHTTPTest', 'DoS attacks-Hulk'],'DoS')

#DDoS	(DDOS attack-LOIC-UDP', DDOS attack-HOIC',DDoS attacks-LOIC-HTTP)
#BOT	(Bot)
#Inf	(Infilteration)
#Web attack	(Brute Force -Web', Brute Force -XSS', SQL Injection')
#BF	(FTP-BruteForce', SSH-Bruteforce')
#DoS (DoS attacks-GoldenEye', DoS attacks-Slowloris', DoS attacks-SlowHTTPTest', DoS attacks-Hulk')

In [10]:
print(X.isna().values.any())
print (pd.unique(y.squeeze()))

X_Train, X_2, y_Train, y_2= train_test_split(X, y, test_size=0.1, random_state=1)
X_Test, X_Val, y_Test, y_Val = train_test_split(X_Train, y_Train, test_size=0.1, random_state=1) 

print(X_Train.isna().values.any())
print(X_Val.isna().values.any())
print(X_Test.isna().values.any())



X_Val.to_csv('X_Val.csv', index = False)
y_Val.to_csv('y_Val.csv', index = False)

X_Test.to_csv('X_Test.csv', index = False)
y_Test.to_csv('y_Test.csv', index = False)

False
['Benign' 'Bot' 'DoS' 'Label' 'Web_attack' 'Bruteforce' 'DDoS'
 'Infilteration']
False
False
False


In [11]:
X_Train.index[np.isinf(X_Train).any(1)]

Int64Index([], dtype='int64')

In [12]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE(sampling_strategy='minority')
print(X_Train.isna().values.any())
print (pd.unique(y_Train.squeeze()))
X_smote, y_smote = oversample.fit_resample(X_Train, y_Train)
print (pd.unique(y_smote.squeeze()))
print(type(oversample))
print (X_smote.shape)
X_smote.to_csv('X_smote.csv', index = False)
y_smote.to_csv('y_smote.csv', index = False)

False
['Benign' 'Bot' 'DDoS' 'DoS' 'Bruteforce' 'Infilteration' 'Web_attack'
 'Label']
['Benign' 'Bot' 'DDoS' 'DoS' 'Bruteforce' 'Infilteration' 'Web_attack'
 'Label']
<class 'imblearn.over_sampling._smote.base.SMOTE'>
(4900912, 78)
